[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gcmatos/python-para-geociencias/blob/master/notebooks/2.2%20EDA%20Registro%20de%20Terremotos%20USGS.ipynb)

# 2.2 Análise Exploratória de Dados
## *Registro USGS de terremotos desde 1965*
#### Fonte: [https://www.kaggle.com/usgs/earthquake-database](https://www.kaggle.com/usgs/earthquake-database)

# Configuração de ambiente

## Instalação de dependências

In [0]:
# Instalação do Basemap e dependências para dados geoespaciais
!apt install proj-bin libproj-dev libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/v1.1.0.tar.gz

## Importar bibliotecas

In [0]:
# Processamento
import numpy as np
import pandas as pd
from pandas import Series

# Visualizacao
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import seaborn as sns

# Filtrar alertas
import warnings
warnings.filterwarnings('ignore')

In [0]:
# Matplotlib em células de códigos
%matplotlib inline

In [0]:
# Matplotlib style
plt.style.use('default')

# Importação dos dados
Subir o arquivo **`earthquakes-database.csv`**

In [0]:
earthquakes = pd.read_csv("earthquakes-database.csv")

# Auditando dados

> Colunas com datas foram reconhecidas como `pandas.to_datetime` durante a importação?

> [Descrição das variáveis na USGS](https://earthquake.usgs.gov/data/comcat/data-eventterms.php)


Trata-se de uma série ou uma tabela de atributos?

In [0]:
# earthquakes.head()
# earthquakes.sample(5)
earthquakes.tail()

Quais são os nomes (*labels*) das variáveis (colunas)?


In [0]:
# earthquakes.columns

for i in earthquakes.columns:
    print(i)

Resumo de informações sobre todas as variáveis presentes no dataset:

In [0]:
earthquakes.info()

Resumo estatístico das variáveis numéricas:

In [0]:
earthquakes.describe()

## Valores nulos

Como estão distribuidos os valores nulos no data set?

In [0]:
earthquakes.isnull().sum()

## Variáveis categóricas

Como é a distribuição das variáveis categóricas?

In [0]:
earthquakes['Type'].value_counts()

In [0]:
# Magnitude Type
earthquakes['Magnitude Type'].value_counts()

## Séries de tempo

As variáveis de tempo foram identificadas corretamente durante a importação?

In [0]:
earthquakes['Date'].dtype

In [0]:
earthquakes['Time'].dtype

É necessário transformar estas variáveis em `datetime`?

In [0]:
earthquakes['Date'] = pd.to_datetime(earthquakes['Date'])

A variável 'Date' pode ser decomposta em uma nova variável 'Year' para realizarmos estatísticas por ano?

In [0]:
# Criar coluna 'Year' a partir da coluna 'Date'
earthquakes['Year'] = earthquakes['Date'].dt.year

In [0]:
# Converter a coluna 'Time' em datetime
earthquakes['Time'] = pd.to_datetime(earthquakes['Time'])

## Filtrar para visualização
Filtrar os tipos de sismos para melhor visualizar os dados.

In [0]:
# Filtrando sismos tectônicos e geológicos
earthquakes_geo = earthquakes[earthquakes['Type'] == 'Earthquake']

In [0]:
# Filtrar sismos nucleares
nuclear_explosion = earthquakes[earthquakes['Type'] == 'Nuclear Explosion']

# Visualização


## Pré-visualização

Se plotarmos todas as variáveis de uma só vez, a visualização seria esclarecedora?

In [0]:
earthquakes.plot();

Podemos visualizar em apenas um gráfico todas as variáveis contínuas e suas relações para se ter um panorama das distribuições e correlações?

In [0]:
sns.pairplot(earthquakes, diag_kind='kde', plot_kws={'alpha': 0.2});

## Distribuição de variáveis

In [0]:
def bar_plot(data):
    series = pd.Series(data).value_counts().sort_index()
    return series.plot(kind='bar')

def line_plot(data):
    series = pd.Series(data).value_counts().sort_index()
    return series.plot()

### Distribuição de valores nulos

In [0]:
earthquakes.isnull().sum().plot.barh();

### Variáveis contínuas

Como é o histograma de distribuição de valores de magnitude, profundidade e etc.?

In [0]:
# Plotar histograma de Magnitude para sismos geológicos

In [0]:
# Plotar histograma de Magnitude para sismos nucleares

In [0]:
earthquakes['Depth Seismic Stations'].hist();

### Variação ao longo do tempo
Como é a frequência do sismos ao longo do tempo?

In [0]:
# earthquakes_geo['Year'].value_counts().sort_index().plot()
line_plot(earthquakes_geo['Year'])
# Plot
plt.ylabel("Frequência")
plt.title("Sismos por ano")
plt.show()

Qual é o maior número de sismos?

In [0]:
earthquakes_geo['Year'].value_counts().max()

Em qual ano se deu a maior ocorrência de terremotos?

In [0]:
earthquakes_geo['Year'].value_counts().idxmax()

Como é a distribuição de valores discretos de magnitude de sismos?


In [0]:
magnitudes_roundup = np.around(earthquakes["Magnitude"])
magnitudes_roundup.value_counts().sort_index().plot.bar()
plt.ylabel("Frequência")
plt.title("Magnitude")
plt.show()

Como fica a distribuição de sismos de explosões nucleares ao longo do tempo?

In [0]:
# Plotar a variação de sismos nucleares no tmepo

## Correlações entre variáveis

Como é a distribuição de sismos por profundidade?

In [0]:
plt.scatter(earthquakes["Magnitude"], earthquakes["Depth"])
plt.ylim(800, -50)
plt.ylabel('Depth (km)')
plt.xlabel('Magnitude')
plt.show()

## Distribuição geoespacial

Como estão distribuídos os sismos do data set em termos de coordenadas geográficas?

In [0]:
m = Basemap(projection='mill', llcrnrlat=-80, urcrnrlat=80, llcrnrlon=-180, urcrnrlon=180, lat_ts=20, resolution='c')

fig = plt.figure(figsize=(16, 24))

longitudes_geo = earthquakes_geo["Longitude"].tolist()
latitudes_geo = earthquakes_geo["Latitude"].tolist()

a, b = m(longitudes_geo, latitudes_geo)

plt.title("Sismos (verde)")
m.plot(a, b, "o", markersize = 3, color = 'green', alpha=0.2)

m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='gray', lake_color='aqua')
m.drawmapboundary()
m.drawcountries()

plt.show()

In [0]:
plt.close('all')

Como é a distribuição dos sismos por zonas de latitude norte, equatorial e sul?

In [0]:
def division(data):
    
    # Split
    north_n = sum(data["Latitude"] >= 30)
    middle_n = sum(np.logical_and(data["Latitude"] < 30, data["Latitude"] > -30))
    south_n = sum(data["Latitude"] <= -30)
    
    # Sumário
    total = north_n + middle_n + south_n
    north_p = north_n / total * 100
    middle_p = middle_n / total * 100
    south_p = south_n / total * 100
    
    return [north_n, middle_n, south_n, north_p, middle_p, south_p]



In [0]:
# Segmentar dados por zona
earthquakes_geo_split = division(earthquakes_geo)

# Criar dataframe e inserir dados de frequência
earthquakes_geo_split_df = pd.DataFrame(earthquakes_geo_split[0:3],
                                        index=['Norte', 'Equatorial', 'Sul'],
                                        columns=['Frequência (n)'])

# Inserir porcentagens no dataframe
earthquakes_geo_split_df['Porcentagem (%)'] = earthquakes_geo_split[3:]

# Visualizar
earthquakes_geo_split_df.iloc[::-1, 1].plot.barh();

Como é a distribuição de sismos a partir de 2012?

In [0]:
# Filtrar sismos entre os anos de 2012 e 2016
last_geo = earthquakes_geo[(earthquakes_geo["Year"] >= 2012) & (earthquakes_geo["Year"] <= 2016)]

# Criar listas de lat&long
longitudes_geo = last_geo["Longitude"].tolist()
latitudes_geo = last_geo["Latitude"].tolist()

m = Basemap(projection='mill', llcrnrlat=-80, urcrnrlat=80, llcrnrlon=-180, urcrnrlon=180, lat_ts=20, resolution='l')
x, y = m(longitudes_geo, latitudes_geo)

plt.figure(figsize=(16, 24))
plt.title("Sismos (verde) entre 2012 e 2016")

m.plot(x, y, "o", markersize = 3, color = 'green')

m.drawcoastlines()
m.fillcontinents(color='gray', lake_color='aqua', alpha=0.6)
m.drawmapboundary()
m.drawcountries()

plt.show()